In [1]:
import numpy as np
import pandas as pd
import calendar
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
cardf= pd.read_csv('./cardata.csv')
cardf.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [3]:
#make a list of subcategories for attribute make
makeCat1= ['Accura','BMW','Ferrari','Jaguar','Lexus','Mecedes','Nisson','Toyota']
makeCat2= ['Chevrolet','Dodge']
makeCat3= ['Pontiac','Saturn']
makeCat4= ['Porche','Saab']

#change the attibutes of make

new_df= cardf.replace('Accura','Honda')
new_df1= new_df.replace('BMW','Honda')
new_df2= new_df1.replace('Ferrari','Honda')
new_df3= new_df2.replace('Jaguar','Honda')
new_df4= new_df3.replace('Lexus','Honda')
new_df5= new_df4.replace('Mecedes','Honda')
new_df6= new_df5.replace('Nisson','Honda')
new_df7= new_df6.replace('Toyota','Honda')
new_df8= new_df7.replace('Mazda','Honda')

new_df9= new_df8.replace('Chevrolet','Ford')
new_df10= new_df9.replace('Dodge','Ford')
new_df11= new_df10.replace('Pontiac','Mercury')
new_df12= new_df11.replace('Saturn','Mercury')
new_df13= new_df12.replace('Porche','VW')
new_df14= new_df13.replace('Saab','VW')

#make a list of subcategories for vehicle price
vehiclePriceCat1= ['20,000 to 29,000','30,000 to 39,000']
vehiclePriceCat2= ['40,000 to 59,000','60,000 to 69,000']

#change the vehicle price
new_df15= new_df14.replace('20,000 to 29,000','20,000 to 39,000')
new_df16= new_df15.replace('30,000 to 39,000','20,000 to 39,000')

new_df17= new_df16.replace('40,000 to 59,000','40,000 to 69,000')
new_df18= new_df17.replace('60,000 to 69,000','40,000 to 69,000')



#make a list of subcategories for Days:Policy Accident
policyAccidentCat1= ['1 to 7','8 to 15']
policyAccidentCat2= ['none']

#change for the Days:Policy-Accident
new_df19= new_df18.replace('1 to 7','1 to 15')
new_df20= new_df19.replace('8 to 15','1 to 15')

new_df21= new_df20.replace('none','more than 30')



#make a list of subcategories for age of vehicle
ageofvehicleCat1= ['new','2 years', '3 years']
ageofvehicleCat2= ['4 years','5 years','6 years']

#change for the Days:Policy-Claim
new_df22= new_df21.replace('new','less than 4 years')
new_df23= new_df22.replace('2 years','less than 4 years')
new_df24= new_df23.replace('3 years','less than 4 years')
new_df25= new_df24.replace('4 years','4 to 6 years')
new_df26= new_df25.replace('5 years','4 to 6 years')
new_df27= new_df26.replace('6 years','4 to 6 years')


#address change claim
addChangeClaimCat1= ['under 6 months', '1 year','2 to 3 years']

#change for the address change claim
new_df28= new_df27.replace('under 6 months','0 to 3 years')
new_df29= new_df28.replace('1 year','0 to 3 years')
new_df30= new_df29.replace('2 to 3 years','0 to 3 years')

#number of vehicle
numOfVehicleCat1= ['2 vehicles','3 to 4','5 to 8','more than 8']

new_df31= new_df30.replace('2 vehicles','more than 1')
new_df32= new_df31.replace('3 to 4','more than 1')
new_df33= new_df32.replace('5 to 8','more than 1')
new_df34= new_df33.replace('more than 8', 'more than 1')



In [4]:
#handle the missing values
new_df35= new_df34.replace(0,17)

In [5]:
new_df35.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,less than 4 years,26 to 30,No,No,External,more than 30,0 to 3 years,more than 1,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,4 to 6 years,31 to 35,Yes,No,External,more than 30,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,more than 30,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Honda,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,4 to 6 years,31 to 35,No,No,External,more than 30,no change,1 vehicle,1994,Collision


In [6]:
new_df35.to_csv('lessSubCategoriesData.csv',index=None,sep='\t')

carCompressedData= new_df35   #gives the new compressed Data...